In [14]:
import torch
import torch.nn as nn
import numpy as np
import random
import torchtext
from tqdm import tqdm
import re
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Load data

In [15]:

p_intent = []
p_snippet = []
c = 0
iit = []
# train data
with open('x.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      # for removing repeated data
      if i[:-1].lower() not in p_intent:
        iit.append(c)
      c+=1
      p_intent.append(i[:-1].lower())
p_intentn = []
# if load noise training set
# with open('x25/x50/x75/x100.txt', 'r',encoding='UTF-8') as intent:
#     for i in intent:
#       p_intentn.append(i[:-1].lower())
with open('x.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      p_intentn.append(i[:-1].lower())

with open('y.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippet.append(i.lower())

train_pairs = [[p_intentn[i],p_snippet[i]] for i in iit]
# validation data
p_intentval = []
p_snippetval = []
iiv = []
c= 0
with open('xv.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      # for removing repeated data and data already in training data
      if i[:-1].lower() not in p_intent and i[:-1].lower() not in p_intentval:
        iiv.append(c)
      
      p_intentval.append(i[:-1].lower())
      c+=1
p_intentvaln = []
with open('xv.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      p_intentvaln.append(i[:-1].lower())
      c+=1
with open('yv.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      #i = i.split(' ')
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      # if i.lower() not in p_snippet:
      p_snippetval.append(i.lower())
val_pairs = [[p_intentvaln[i],p_snippetval[i]] for i in iiv]
#  test data
p_intenttest = []
p_snippettest = []
ii = []
c= 0
with open('xt.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      # for removing repeated data and data already in training data
      if i[:-1].lower() not in p_intent and i[:-1].lower() not in p_intenttest:
        ii.append(c)
      c+=1
      p_intenttest.append(i[:-1].lower())
with open('yt.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      # if i.lower() not in p_snippet:
      p_snippettest.append(i.lower())
test_pairs = [[p_intenttest[i],p_snippettest[i]] for i in ii]
# lv1 noise test data
p_intenttest1 = []
p_snippettest1 = []
with open('xt1.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      
      p_intenttest1.append(i[:-1].lower())
with open('yt1.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippettest1.append(i.lower())
test_pairs1 = [[p_intenttest1[i],p_snippettest1[i]] for i in ii]
# lv2 noise test data
p_intenttest2 = []
p_snippettest2 = []
with open('xt2.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      
      p_intenttest2.append(i[:-1].lower())
with open('yt2.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      p_snippettest2.append(i.lower())
test_pairs2 = [[p_intenttest2[i],p_snippettest2[i]] for i in ii]

In [16]:
# if load extra training data
# p_intentextra = []
# p_snippetextra = []
# c = 0
# iite = []
# with open('xextra.txt', 'r',encoding='UTF-8') as intent:
#     for i in intent:
#       if i[:-1].lower() not in p_intentextra and i[:-1].lower() not in p_intent and i[:-1].lower() not in p_intenttest and i[:-1].lower() not in p_intentval:
#         iite.append(c)
#       c+=1
#       p_intentextra.append(i[:-1].lower())

# with open('yextra.txt', 'r',encoding='UTF-8') as snippet:
#     for i in snippet:
#       p_snippetextra.append(i[:-1].lower())
# extratrain = [[p_intentextra[i],p_snippetextra[i]] for i in iite]
# train_pairs = train_pairs+extratrain

## Pre-processing and building vocab

In [17]:

SRC_TEXT = torchtext.legacy.data.Field(sequential=True,
                              
                                )
TARG_TEXT = torchtext.legacy.data.Field(sequential=True,
                                 )
                                 

In [18]:
ptrainx = [['sos']+x[0].split()+['eos'] for x in train_pairs]
ptrainy = [['sos']+x[1].split()+['eos'] for x in train_pairs]
pvalx = [['sos']+x[0].split()+['eos'] for x in val_pairs]
pvaly = [['sos']+x[1].split()+['eos'] for x in val_pairs]
ptestx = [['sos']+x[0].split()+['eos'] for x in test_pairs]
ptesty = [['sos']+x[1].split()+['eos'] for x in test_pairs]
ptestx1 = [['sos']+x[0].split()+['eos'] for x in test_pairs1]
ptesty1 = [['sos']+x[1].split()+['eos'] for x in test_pairs1]
ptestx2 = [['sos']+x[0].split()+['eos'] for x in test_pairs2]
ptesty2 = [['sos']+x[1].split()+['eos'] for x in test_pairs2]

In [19]:
SRC_TEXT.build_vocab(ptrainx) 
print(len(SRC_TEXT.vocab))
print(SRC_TEXT.vocab.itos[0])
print(SRC_TEXT.vocab.itos[1])
print(SRC_TEXT.vocab.itos[2])
print(SRC_TEXT.vocab.itos[3])
print(SRC_TEXT.vocab.stoi['sos'])
print(SRC_TEXT.vocab.stoi['eos'])

TARG_TEXT.build_vocab(ptrainy)
print(len(TARG_TEXT.vocab))
print(TARG_TEXT.vocab.itos[0])
print(TARG_TEXT.vocab.itos[1])
print(TARG_TEXT.vocab.itos[2])
print(TARG_TEXT.vocab.itos[3])
print(TARG_TEXT.vocab.stoi['sos'])
print(TARG_TEXT.vocab.stoi['eos'])

18911
<unk>
<pad>
eos
sos
3
2
19063
<unk>
<pad>
(
)
6
5


In [20]:
final_trainx = [torch.tensor([SRC_TEXT.vocab.stoi[x] for x in i]) for i in ptrainx]
final_trainy = [torch.tensor([TARG_TEXT.vocab.stoi[x] for x in i]) for i in ptrainy]
final_valx = [torch.tensor([SRC_TEXT.vocab.stoi[x] for x in i]) for i in pvalx]
final_valy = [torch.tensor([TARG_TEXT.vocab.stoi[x] for x in i]) for i in pvaly]

In [21]:
final_testx = [torch.tensor([SRC_TEXT.vocab.stoi[x] for x in i]) for i in ptestx]
final_testy = [torch.tensor([TARG_TEXT.vocab.stoi[x] for x in i]) for i in ptesty]
final_testx1 = [torch.tensor([SRC_TEXT.vocab.stoi[x] for x in i]) for i in ptestx1]
final_testy1 = [torch.tensor([TARG_TEXT.vocab.stoi[x] for x in i]) for i in ptesty1]
final_testx2 = [torch.tensor([SRC_TEXT.vocab.stoi[x] for x in i]) for i in ptestx2]
final_testy2 = [torch.tensor([TARG_TEXT.vocab.stoi[x] for x in i]) for i in ptesty2]

## Create dataset

In [22]:
data = list(zip(final_trainx,final_trainy))
datav = list(zip(final_valx,final_valy))
datat = list(zip(final_testx,final_testy))
datat1 = list(zip(final_testx1,final_testy1))
datat2 = list(zip(final_testx2,final_testy2))

In [23]:
# for padding in the mini-batch
def collate_fn(data):
    inputs,targ = zip(*data)
    seq_len = torch.tensor([len(i) for i in inputs])
    inputs= torch.nn.utils.rnn.pad_sequence(inputs, padding_value=1)
    targ = torch.nn.utils.rnn.pad_sequence(targ, padding_value=1)

    return inputs,targ,seq_len
d = torch.utils.data.DataLoader(datat, batch_size = 64 ,shuffle = False,collate_fn= collate_fn)

In [24]:
for i,(inp,tar,h) in enumerate(d):
  print(tar)
  print([SRC_TEXT.vocab.itos[i] for i in inp[:,1]])
  print(tar.size(1))
  break

tensor([[    6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6,     6,     6,     6,     6,     6,     6,
             6,     6,     6,     6],
        [   13,    13,    13,    13,    13,    13,    13,    13,    13,    13,
            13,    13,    13,    13,    13,    13,    13,    13,    13,    13,
            13,    13,    13,    13,    13,    13,    13,    13,    13,    13,
            13,    13,    13,    13,    13,    13,    13,    13,    13,    13,
            13,    13,    13,    13,    13,    13,    13,    13,    13,    13,
            13,    13,    13,    13,    13,    13,    13,    13,    13,    13,
            13

## Encoder,Decoder, seq2seq

In [25]:
class Encoder(nn.Module):
    def __init__(self, input_size,embedding_size,hidden_size,num_layers,dropout_rate):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size,num_layers , bidirectional = True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size*2, hidden_size)
        self.r = torch.nn.ReLU()
    def forward(self, input,seq_length):
        embedded = self.dropout(self.embedding(input))
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, seq_length,enforce_sorted=False)
        output, hidden = self.gru(packed)
        output,_ = nn.utils.rnn.pad_packed_sequence(output,batch_first=False,total_length=max(seq_length),)
        # concat two hiddens to one hidden for decoder use
        hidden = torch.cat((hidden[-1],hidden[-2]),dim=1)
        hidden = self.r(self.fc(hidden)).unsqueeze(0)
        return output,hidden


In [26]:
class Decoder(nn.Module):
    def __init__(self, input_size,embedding_size,hidden_size,output_size,num_layers,dropout_rate):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout_rate)
        self.embedding = nn.Embedding(input_size,embedding_size)
        self.gru = nn.GRU(hidden_size*2+embedding_size, hidden_size,num_layers,dropout=dropout_rate)
        self.attension = nn.Linear(hidden_size*3,1)
        self.out = nn.Linear(hidden_size, output_size)
        self.relu = nn.Tanh()
        self.softmax = nn.Softmax(dim=0)
    def forward(self, encoder_output ,input, hidden):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        h_attension = hidden.repeat(encoder_output.size(0),1,1)
        # calculate attention
        attention = self.relu(self.attension(torch.cat((h_attension,encoder_output),dim=2)))
        attention = self.softmax(attention)
        c = torch.bmm(attention.permute(1,2,0),encoder_output.permute(1,0,2)).permute(1,0,2)
        embedded = torch.cat((embedded,c),dim=2)
        # print(embedded.shape)
        output,hidden = self.gru(embedded,hidden)
        pred = self.out(output)
        pred = pred.squeeze(0)
        return pred,hidden

In [27]:

class Seq2Seq(nn.Module):
    def __init__(self, encoder,decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    def forward(self, source,target,s,teacher_force_ratio=0.5,):
        batch_size = source.size(1)
        target_length = target.size(0)
        target_vocab_size = len(TARG_TEXT.vocab)
        output_encoder,hidden = self.encoder(source,s)
        outputs = torch.zeros(target_length,batch_size,target_vocab_size).to(device)
        x = target[0,:]
        # print(hidden)
        for t in range(1,target_length):
          output,hidden = self.decoder(output_encoder,x,hidden)
          ### output: batch*vocab_size
          outputs[t] = output
          pred_max = output.argmax(1)
          x = target[t] if random.random() < teacher_force_ratio else pred_max
        return outputs

In [28]:
def train(datat,batch_s,model):
  model.train()
  d = torch.utils.data.DataLoader(datat, batch_size = batch_s ,shuffle = True,collate_fn= collate_fn)
  loss_train = 0
  acc_train = 0
  for i, (x_train,x_label,s) in enumerate(d,start = 1):
      optimizer.zero_grad()
      x_train = x_train.to(device)
      x_label = x_label.to(device)
      predict = model(x_train,x_label,s)
      pred_dim = predict.shape[-1]
      x_label = x_label[1:].view(-1)
      predict = predict[1:].view(-1, pred_dim)
      # print(x_label.shape)
      loss = criterion(predict, x_label)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
      _pred = predict.argmax(dim=-1)  
      corrects = _pred.eq(x_label) 
      mask = torch.logical_not(x_label.eq(1)) 
      corrects *= mask
      acc_train += corrects.sum().float() / mask.sum()
  return loss_train/i,acc_train/i

def val(datat,batch_s,model):
  model.eval()
  d = torch.utils.data.DataLoader(datat, batch_size = batch_s ,shuffle = False,collate_fn= collate_fn)
  loss_val = 0
  acc_val = 0
  for i, (x_val,x_label,s) in enumerate(d,start = 1):
      with torch.no_grad():
        x_val = x_val.to(device)
        x_label = x_label.to(device)
        predict = model(x_val,x_label,s)
        pred_dim = predict.shape[-1]
        x_label = x_label[1:].view(-1)
        predict = predict[1:].view(-1, pred_dim)

        loss = criterion(predict, x_label)
    
        loss_val += loss.item()
        _pred = predict.argmax(dim=-1)  # [b, targ_seq_len, target_vocab_size]=>[b, targ_seq_len]
        corrects = _pred.eq(x_label) 
        mask = torch.logical_not(x_label.eq(1)) 
        corrects *= mask
      # print(corrects.shape)
      # print(x_label.shape)
        acc_val += corrects.sum().float() / mask.sum()
        # corrects *= mask
      # print(mask.sum())
      # print(corrects.shape)
      # print(x_label.shape)
        # acc_val += corrects.sum().float() / (x_label.shape[0])
  return loss_val/i,acc_val/i

In [29]:
def predict(sen,trg, model, source_vocab, target_vocab ):

    # Convert each source token to integer values using the vocabulary
    tokens =  ['sos']+[token.lower() for token in sen.split(' ')] + ['eos']
    src_indexes = [source_vocab.stoi[token] for token in tokens]
    # print(src_indexes)
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    # print(src_tensor.shape)
    model.eval()

    # Run the forward pass of the encoder
    encoder_outputs, hidden = model.encoder(src_tensor,torch.tensor([src_tensor.shape[0]]))

    # Take the integer value of <sos> from the target vocabulary.
    trg_index = [target_vocab.stoi['sos']]
    next_token = torch.LongTensor(trg_index).to(device)

    # print(next_token)
    trg_indexes = [trg_index[0]]

    # outputs = []
    with torch.no_grad():
        # Use the hidden and cell vector of the Encoder and in loop
        # run the forward pass of the OneStepDecoder until some specified
        # step (say 50) or when <eos> has been generated by the model.
        for i in range(200):
            output, hidden = model.decoder(encoder_outputs, next_token, hidden)


            # Take the most probable word
            
            next_token = output.argmax(1)

            trg_indexes.append(next_token.item())

            predicted = target_vocab.itos[next_token[0]]
            # print(predicted)
            if predicted == 'eos':
                break
            
    predicted_words = [target_vocab.itos[i] for i in trg_indexes]

    return predicted_words[1:-1]


## Model setting and train model

In [30]:
### model parameter
input_size = len(SRC_TEXT.vocab)
output_size = len(TARG_TEXT.vocab)
encoder_embedding_size = 400
decoder_embedding_size = 400
hidden_size = 800
number_layers = 1
encoder_dropout_rate = 0.25
decoder_dropout_rate = 0.25
###
enc = Encoder(input_size, encoder_embedding_size, hidden_size, number_layers, encoder_dropout_rate).to(device)
dec = Decoder(output_size, decoder_embedding_size, hidden_size,output_size,number_layers, decoder_dropout_rate).to(device)

model = Seq2Seq(enc, dec).to(device)
criterion = nn.CrossEntropyLoss(ignore_index = 1)

optimizer = torch.optim.Adam(model.parameters(),lr = 0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer , 8,
                gamma =0.9)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [32]:
import timeit
early_ = []
for e in range(100):
    # print('epoch: ' + str(e))
    start = timeit.default_timer()
    t = train(data,256,model)
    stop = timeit.default_timer()
    print('Time: ', stop - start) 
    v = val(datav,256,model)
    test = val(datat,256,model)
    test1,test2 = [0,0],[0,0]
    test1 = val(datat1,256,model)
    test2 = val(datat2,256,model)
    prediction = []
    b = [0]
    # early stop
    if len(early_)>10:
      if v[0]>=early_[e-10]:
        print('EPOCH = {} loss: {:.3f}, {}: {:.3f},val_loss: {:.3f}, val_{}: {:.3f},test_loss: {:.3f}, test_{}: {:.3f}, test1_loss: {:.3f},test1_{}: {:.3f},test2_loss: {:.3f},test2_{}: {:.3f}'.format(
            e, t[0], "acc", t[1], v[0], "acc", v[1],test[0], "acc", test[1],test1[0], "acc", test1[1],test2[0], "acc", test2[1]))
        break
    early_.append(v[0])
    scheduler.step()
    print('EPOCH = {} loss: {:.3f}, {}: {:.3f},val_loss: {:.3f}, val_{}: {:.3f},test_loss: {:.3f}, test_{}: {:.3f}, test1_loss: {:.3f},test1_{}: {:.3f},test2_loss: {:.3f},test2_{}: {:.3f}'.format(
            e, t[0], "acc", t[1], v[0], "acc", v[1],test[0], "acc", test[1],test1[0], "acc", test1[1],test2[0], "acc", test2[1]))

In [ ]:
# calcualte BLEU for test set
all_re = []
gt= [[i[1].split()]for i in test_pairs]
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method4
total = 0
for i in tqdm(range(len(test_pairs))):
       
      
        references = gt[i]
        
        candidates = predict(test_pairs[i][0],test_pairs[i][1], model, SRC_TEXT.vocab, TARG_TEXT.vocab)

        sbs = sentence_bleu(references, candidates,smoothing_function=smoothie)
        all_re.append(sbs)
        total+=sbs
total*100/len(test_pairs)

In [ ]:
# calcualte BLEU for lv1 noise test set
gt= [[i[1].split()]for i in test_pairs1]
total = 0
for i in tqdm(range(len(test_pairs))):
       
      
        references = gt[i]
      
        candidates = predict(test_pairs1[i][0],test_pairs1[i][1], model, SRC_TEXT.vocab, TARG_TEXT.vocab)



        total+=sentence_bleu(references, candidates,smoothing_function=smoothie)
total*100/len(test_pairs)

In [ ]:
# calcualte BLEU for lv2 noise test set
gt= [[i[1].split()]for i in test_pairs2]
total = 0
for i in tqdm(range(len(test_pairs))):
       
      
        references = gt[i]
      
        candidates = predict(test_pairs2[i][0],test_pairs2[i][1], model, SRC_TEXT.vocab, TARG_TEXT.vocab)



        total+=sentence_bleu(references, candidates,smoothing_function=smoothie)
total*100/len(test_pairs)

In [ ]:
# calcualte BLEU for negation test set
ngx = []
ngy = []
with open('xneg.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      ngx.append(i[:-1].lower())
with open('yneg.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      # if i.lower() not in p_snippet:
      ngy.append(i.lower())
ngpairs = [[ngx[i],ngy[i]] for i in range(len(ngy))]
gt= [[i[1].split()]for i in ngpairs]
total = 0
for i in tqdm(range(len(ngpairs))):
       
      
        references = gt[i]
      
        candidates = predict(ngpairs[i][0],ngpairs[i][1], model, SRC_TEXT.vocab, TARG_TEXT.vocab)



        total+=sentence_bleu(references, candidates,smoothing_function=smoothie)
total*100/len(ngpairs)

In [ ]:
# calcualte BLEU for non-negation test set
nongx = []
nongy = []
with open('xnoneg.txt', 'r',encoding='UTF-8') as intent:
    for i in intent:
      nongx.append(i[:-1].lower())
with open('ynoneg.txt', 'r',encoding='UTF-8') as snippet:
    for i in snippet:
      i = i[:-1]
      if i[-1]==' ':
        i=i[:-1]
      # if i.lower() not in p_snippet:
      nongy.append(i.lower())
nongpairs = [[nongx[i],nongy[i]] for i in range(len(nongy))]
gt= [[i[1].split()]for i in nongpairs]
total = 0
for i in tqdm(range(len(nongpairs))):
       
      
        references = gt[i]
      
        candidates = predict(nongpairs[i][0],nongpairs[i][1], model, SRC_TEXT.vocab, TARG_TEXT.vocab)



        total+=sentence_bleu(references, candidates,smoothing_function=smoothie)
total*100/len(nongpairs)